In [1]:
!pip install -U 'tensorflow[and-cuda]'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 27.9 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 99.3 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 296.8/296.8 MB 6.1 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 85.8 MB/s eta 0:00:00ta 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 620.6/620.6 MB 2.7 MB/s eta 0:00:00:00:0100:01
  Attempting uninstall: nvidia-nccl-cu12
    Found existing installation: nvidia-nccl-cu12 2.21.5
    Uninstalling nvidia-nccl-cu12-2.21.5:
      Successfully uninstalled nvidia-nccl-cu12-2.21.5
  Attempting uninstall: ml_dtypes
    Found existing installation: ml-dtypes 0.4.1
    Uninstalling ml-dtypes-0.4.1:
      Successfully uninstalled ml-dtypes-0.4.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.18.0
    Uninstalling tensorboard-2.18.0:
      Successfully uninstalled tensorboard-2.18.0
  Attempting uninstall: kera

In [2]:
!pip install evaluate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 3.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 4.3 MB/s eta 0:00:000:00:0100:01
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 19.0.1
    Uninstalling pyarrow-19.0.1:
      Successfully uninstalled pyarrow-19.0.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
bigframes 2.12.0 requires google-cloud-bigquery-storage<3.0.0,>=2.30.0, which is not installed.
pylibcudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
cudf-cu12 25.2.2 requires pyarrow<20.0.0a0,>=14.0.0; platform_machine == "x86_64", but you have pyarrow 22.0.0 which is incompatible.
bigframes 2.12.0 requires rich<14,>=12.4.4, but you have rich 14.2.0 which is incompatible.
cudf-polars-cu12 25.6.0 requires pyli

# **1.   Imports**

In [3]:
import torch
import gc
import random
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import evaluate
import time
import glob
import ast

from typing import Dict, Any, List, Optional, Tuple, cast
from dataclasses import dataclass, asdict

# Hugging Face Libraries
from datasets import load_dataset, DatasetDict
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    TrainingArguments,
    Trainer,
    DataCollatorWithPadding,
    EvalPrediction
)
from peft import LoraConfig, TaskType, get_peft_model

In [4]:
# Hardware & Reproducibility
SEED = 42

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [5]:
# Data & Model Settings
MODEL_NAME = "distilbert-base-uncased"
TRAIN_SAMPLE_SIZE = 3000
NUM_LABELS = 6
MAX_LENGTH = 128

In [6]:
# PSO Settings
PSO_POPULATION_SIZE = 20   # Number of particles
PSO_EPOCHS = 5            # Number of iterations
INERTIA_W = 0.7           # Inertia weight
COGNITIVE_C1 = 1.5        # Personal best weight
SOCIAL_C2 = 1.5           # Global best weight

In [7]:
# Search Space (Discrete Options)
WARMUP_OPTIONS = [0.0, 0.06, 0.1]
RANK_OPTIONS = [2, 4, 8, 16, 24]
ALPHA_OPTIONS = [8, 16, 32, 64, 96]
DROPOUT_OPTIONS = [0.0, 0.05, 0.1, 0.2]
TARGET_MODULE_OPTIONS = [
    ["q_lin", "v_lin"],                           # Index 0
    ["q_lin", "v_lin", "ffn.lin1", "ffn.lin2"]    # Index 1
]

# **2.   Data Structure Definitions**

In [8]:
@dataclass(frozen=True)
class LoraHyperparameters:
    """Structure to hold a specific set of hyperparameters"""
    learning_rate: float
    warmup_ratio: float
    rank: int
    alpha: int
    dropout: float
    target_modules: List[str]

class Particle:
    """Represents a single particle in the swarm"""
    def __init__(self, dim, min_bound, max_bound):
        self.position = np.random.uniform(min_bound, max_bound, dim)
        self.velocity = np.random.uniform(-1, 1, dim)
        self.best_position = np.copy(self.position)
        self.best_score = -float('inf') # Maximizing accuracy

# **3.   Helper Functions**

In [9]:
def set_global_seed(seed: int):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)

def cleanup_memory():
    """Forcefully releases GPU memory"""
    torch.cuda.empty_cache()
    gc.collect()

# **4.   Data Management Loss**

In [10]:
class DataManager:
    def __init__(self, model_name: str = MODEL_NAME):
        self.tokenizer = AutoTokenizer.from_pretrained(model_name)
        self.dataset: Optional[Dict[str, Any]] = None

    def prepare_data(self) -> Dict[str, Any]:
        if self.dataset is not None:
            return self.dataset

        print("Loading and processing data...")
        full_dataset = cast(DatasetDict, load_dataset("dair-ai/emotion"))

        # Consistent subset selection
        train_subset = full_dataset["train"].shuffle(seed=SEED).select(range(TRAIN_SAMPLE_SIZE))

        def _tokenize(examples):
            return self.tokenizer(
                examples["text"],
                truncation=True,
                padding="max_length",
                max_length=MAX_LENGTH
            )

        tokenized_train = train_subset.map(_tokenize, batched=True)
        tokenized_val = full_dataset["validation"].map(_tokenize, batched=True)

        self.dataset = {
            "train": tokenized_train,
            "validation": tokenized_val,
            "tokenizer": self.tokenizer,
            "num_labels": NUM_LABELS
        }
        print("Data preparation complete.")
        return self.dataset

# **5.   PSO Optimiser Engine**

In [11]:
class PSO_HyperparameterOptimizer:
    def __init__(self, data_bundle: Dict[str, Any]):
        self.data = data_bundle
        self.results: List[Dict[str, Any]] = []
        self.metric = evaluate.load("accuracy")

        # 6 Dimensions: [LR, Warmup, Rank, Alpha, Dropout, Modules]
        # We use indices for discrete lists, continuous for LR
        self.min_bounds = np.array([1e-5, 0, 0, 0, 0, 0], dtype=float)
        self.max_bounds = np.array([
            2e-4,
            len(WARMUP_OPTIONS) - 0.01,
            len(RANK_OPTIONS) - 0.01,
            len(ALPHA_OPTIONS) - 0.01,
            len(DROPOUT_OPTIONS) - 0.01,
            len(TARGET_MODULE_OPTIONS) - 0.01
        ], dtype=float)
        self.dim = 6
        self.best_params = None
        self.best_val_accuracy = -1.0
        self.final_results = [] # Store final results for plotting

    def _map_position_to_params(self, position: np.ndarray) -> LoraHyperparameters:
        """Maps continuous particle position to discrete hyperparameters"""
        # Clip to ensure bounds
        pos = np.clip(position, self.min_bounds, self.max_bounds)
        return LoraHyperparameters(
            learning_rate=float(pos[0]),
            warmup_ratio=WARMUP_OPTIONS[int(pos[1])],
            rank=RANK_OPTIONS[int(pos[2])],
            alpha=ALPHA_OPTIONS[int(pos[3])],
            dropout=DROPOUT_OPTIONS[int(pos[4])],
            target_modules=TARGET_MODULE_OPTIONS[int(pos[5])]
        )

    def _compute_metrics(self, eval_pred: EvalPrediction):
        preds, labels = eval_pred
        preds = np.argmax(preds, axis=1)
        return self.metric.compute(predictions=preds, references=labels)

    def train_model(self, trial_id: int, params: LoraHyperparameters) -> float:
        """Runs a single training trial"""
        print(f"   > Params: LR={params.learning_rate:.2e}, Rank={params.rank}, Alpha={params.alpha}")
        
        model = AutoModelForSequenceClassification.from_pretrained(
            MODEL_NAME, num_labels=self.data["num_labels"]
        )

        peft_config = LoraConfig(
            task_type=TaskType.SEQ_CLS,
            r=params.rank,
            lora_alpha=params.alpha,
            lora_dropout=params.dropout,
            target_modules=params.target_modules
        )
        model = get_peft_model(model, peft_config)

        current_seed = SEED + trial_id
        
        args = TrainingArguments(
            output_dir=f"./results/trial_{trial_id}",
            learning_rate=params.learning_rate,
            per_device_train_batch_size=16,
            per_device_eval_batch_size=16,
            num_train_epochs=3,
            warmup_ratio=params.warmup_ratio,
            weight_decay=0.01,
            eval_strategy="epoch",
            save_strategy="no",
            logging_strategy="epoch",
            seed=current_seed,
            report_to="none",
            load_best_model_at_end=False
        )

        data_collator = DataCollatorWithPadding(tokenizer=self.data["tokenizer"])

        trainer = Trainer(
            model=model,
            args=args,
            train_dataset=self.data["train"],
            eval_dataset=self.data["validation"],
            data_collator=data_collator,
            compute_metrics=self._compute_metrics
        )
        trainer.train()
        
        eval_results = trainer.evaluate()
        log_history = trainer.state.log_history
        
        del model
        del trainer
        cleanup_memory()

        return eval_results["eval_accuracy"], log_history
            
    def run_optimization(self):
        print(f"Starting PSO: {PSO_POPULATION_SIZE} particles, {PSO_EPOCHS} epochs.")
        swarm = [Particle(self.dim, self.min_bounds, self.max_bounds) for _ in range(PSO_POPULATION_SIZE)]
        global_best_pos = np.zeros(self.dim)
        global_best_score = -float('inf')
        trial_count = 0

        for epoch in range(PSO_EPOCHS):
            print(f"\n=== PSO EPOCH {epoch + 1}/{PSO_EPOCHS} ===")
            for i, particle in enumerate(swarm):
                trial_count += 1

                try:
                    params = self._map_position_to_params(particle.position)
                    
                    accuracy, _ = self.train_model(trial_count, params)
                    
                    print(f"   > [Trial {trial_count}] Accuracy: {accuracy:.4%}")

                    if accuracy > particle.best_score:
                        particle.best_score = accuracy
                        particle.best_position = np.copy(particle.position)

                    if accuracy > global_best_score:
                        global_best_score = accuracy
                        global_best_pos = np.copy(particle.position)
                        print(f"   >>> New Swarm Best: {accuracy:.4%}")

                    record = asdict(params)
                    record.update({
                        "trial_id": trial_count, 
                        "val_accuracy": accuracy, 
                        "pso_epoch": epoch+1
                    })
                    self.results.append(record)

                except Exception as e:
                    print(f"!!! ERROR in Trial {trial_count}: {e}")
                    cleanup_memory()

            # Update Particles
            for particle in swarm:
                r1, r2 = np.random.rand(self.dim), np.random.rand(self.dim)
                vel_cognitive = COGNITIVE_C1 * r1 * (particle.best_position - particle.position)
                vel_social = SOCIAL_C2 * r2 * (global_best_pos - particle.position)
                particle.velocity = (INERTIA_W * particle.velocity) + vel_cognitive + vel_social
                particle.position += particle.velocity
                particle.position = np.clip(particle.position, self.min_bounds, self.max_bounds)
                
        # Save Initial Results to CSV immediately after loop
        self.save_results("initial_results.csv")

    def evaluate_top_solutions_with_seeds(self, run_id=1, num_top=5, num_seeds=3):
        """
        Retrains the top 5 unique configs found in PSO 3 times each with NEW seeds.
        Calculates Mean, Median, and Std Dev.
        """
        print(f"\n{'='*60}")
        print(f"ROBUSTNESS CHECK: TOP {num_top} CONFIGS x {num_seeds} NEW SEEDS")
        print(f"{'='*60}")

        # 1. Filter Top Unique Candidates
        sorted_results = sorted(self.results, key=lambda x: x['val_accuracy'], reverse=True)
        unique_candidates = []
        seen_configs = set()

        for res in sorted_results:
            config_key = (
                res['learning_rate'], res['warmup_ratio'], res['rank'], 
                res['alpha'], res['dropout'], tuple(res['target_modules'])
            )
            if config_key not in seen_configs:
                seen_configs.add(config_key)
                unique_candidates.append(res)
            if len(unique_candidates) >= num_top:
                break

        self.final_results = []
        raw_data = []
        
        # 2. Retrain Candidates
        # Renamed loop variable to 'sol_rank' to avoid conflict with LoRA 'rank' parameter
        for sol_rank, candidate in enumerate(unique_candidates, 1):
            params = LoraHyperparameters(
                learning_rate=candidate['learning_rate'],
                warmup_ratio=candidate['warmup_ratio'],
                rank=candidate['rank'],
                alpha=candidate['alpha'],
                dropout=candidate['dropout'],
                target_modules=candidate['target_modules']
            )

            print(f"\n--- Solution Rank {sol_rank} (Original Acc: {candidate['val_accuracy']:.4%}) ---")
            seed_accuracies = []
            #seed_histories = []

            for seed_i in range(num_seeds):
                robust_trial_id = 10000 + (run_id * 100) + seed_i
                display_seed = SEED + robust_trial_id
                
                print(f"   > Retraining Seed {seed_i + 1}/{num_seeds} (Seed: {display_seed})...", end=" ", flush=True)
                
                acc, _ = self.train_model(robust_trial_id, params)
                
                seed_accuracies.append(acc)
                print(f"Acc: {acc:.4%}")
                
                raw_record = asdict(params)
                raw_record.update({
                    "candidate_rank": sol_rank,
                    "seed_index": seed_i + 1,
                    "trial_id": robust_trial_id,
                    "val_accuracy": acc,
                    "original_pso_accuracy": candidate['val_accuracy']
                })
                raw_data.append(raw_record)

            # --- Calculate Statistics ---
            mean_acc = np.mean(seed_accuracies)
            std_acc = np.std(seed_accuracies)

            print(f"   >>> Stats | Mean Acc: {mean_acc:.4%} | Std: {std_acc:.4f}")
            
            # Prepare Record
            # asdict(params) contains keys like 'learning_rate', 'rank', etc.
            record = asdict(params) 
            record.update({
                'solution_rank': sol_rank,
                'original_pso_accuracy': candidate['val_accuracy'],
                'mean_robust_accuracy': mean_acc,
                'std_robust_accuracy': std_acc,
                'seed_accuracies': str(seed_accuracies)
            })
            self.final_results.append(record)

        # 3. Save Results using the helper method
        robust_filename = f"robust_results.csv"
        self.save_top_solutions_results(self.final_results, robust_filename)

        # 4. Save Raw Trials (The 15 rows file)
        raw_df = pd.DataFrame(raw_data)
        raw_df.to_csv("robust_trials.csv", index=False)
        print(f"Raw robustness trials saved to robust_trials.csv")
        
        # 5. Update Best Params based on MEAN accuracy (Stability wins)
        self.final_results.sort(key=lambda x: x['mean_robust_accuracy'], reverse=True)
        if self.final_results:
            winner = self.final_results[0]
            # Print update message (Summary is handled in save_top_solutions_results)
            print(f"\nUpdating Best Params to Solution Rank {winner['solution_rank']}")
            
            self.best_params = LoraHyperparameters(
                learning_rate=winner['learning_rate'],
                warmup_ratio=winner['warmup_ratio'],
                rank=winner['rank'],
                alpha=winner['alpha'],
                dropout=winner['dropout'],
                target_modules=winner['target_modules']
            )
            self.best_val_accuracy = winner['mean_robust_accuracy']
    
    def save_top_solutions_results(self, final_results, filename: str):
        """
        Saves results matching the specific CSV format:
        rank | learning_rate | ... | mean_accuracy | std_accuracy | seed_1 | seed_2 ...
        """
        if not final_results:
            print("No top solutions results to save.")
            return
        
        rows = []
        for result in final_results:
            # Handle Target Modules
            t_mods = result.get('target_modules', [])
            if isinstance(t_mods, str):
                try: t_mods = ast.literal_eval(t_mods)
                except: pass
            
            target_binary = 0 if len(t_mods) <= 2 else 1

            # Build Row
            row = {
                'rank': result.get('solution_rank'),
                'learning_rate': result.get('learning_rate'),
                'warmup_ratio': result.get('warmup_ratio'),
                'rank_r': result.get('rank'),
                'alpha': result.get('alpha'),
                'dropout': result.get('dropout'),
                'target_modules': target_binary,
                'original_accuracy': result.get('original_pso_accuracy'),
                'mean_accuracy': result.get('mean_robust_accuracy'),
                'std_accuracy': result.get('std_robust_accuracy')
            }
            
            # Handle Seed Accuracies
            seeds = result.get('seed_accuracies', [])
            if isinstance(seeds, str):
                try: seeds = ast.literal_eval(seeds)
                except: seeds = []
            
            for i, acc in enumerate(seeds, 1):
                row[f'seed_{i}_accuracy'] = acc
            
            rows.append(row)
        
        df = pd.DataFrame(rows)
        
        # Enforce Column Order (Median Removed)
        cols_order = [
            'rank', 'learning_rate', 'warmup_ratio', 'rank_r', 'alpha', 'dropout', 
            'target_modules', 'original_accuracy', 'mean_accuracy', 'std_accuracy'
        ]
        
        seed_cols = [c for c in df.columns if c.startswith('seed_')]
        cols_order.extend(sorted(seed_cols))
        
        final_cols = [c for c in cols_order if c in df.columns]
        df = df[final_cols]

        df.to_csv(filename, index=False)
        print(f"\nTop solutions results saved to {filename}")
        
        if not df.empty:
            best = df.loc[df['mean_accuracy'].idxmax()]
            print("\n" + "="*60)
            print("BEST CONFIGURATION (Formatted Output):")
            print("="*60)
            print(f"Rank: {best['rank']}")
            print(f"Mean Accuracy: {best['mean_accuracy']:.4%} ± {best['std_accuracy']:.4f}")
            print(f"Params: LR={best['learning_rate']:.2e}, r={best['rank_r']}, alpha={best['alpha']}")
            print("="*60)
    
    def save_results(self, filename: str):
        if not self.results:
            print("No results to save.")
            return
        df = pd.DataFrame(self.results)
        df.to_csv(filename, index=False)
        print(f"Results saved to {filename}")
        
        best_run = df.loc[df['val_accuracy'].idxmax()]
        print("\nBEST VALIDATION RESULT:")
        print(f"Accuracy: {best_run['val_accuracy']:.4%}")
        
        print(f"Rank: {best_run['rank']}, Alpha: {best_run['alpha']}, LR: {best_run['learning_rate']:.2e}")

# **6.   Main Execution Pipeline**

In [12]:
def run_single_optimization(run_id: int, data_bundle: Dict[str, Any]) -> Tuple[float, float, LoraHyperparameters, Any]:
    print(f"\n{'='*60}\nRUN {run_id + 1}\n{'='*60}")

    optimizer = PSO_HyperparameterOptimizer(data_bundle)

    start_time = time.time()
    
    # 1. Broad Search (20 particles)
    optimizer.run_optimization()

    end_time = time.time()
    elapsed_time = (end_time - start_time) / 60 
    
    print("\n" + "="*60)
    print(f"ELAPSED TIME: {elapsed_time:.2f} m")
    print("\n" + "="*60)
    
    optimizer.evaluate_top_solutions_with_seeds(run_id=run_id, num_top=5, num_seeds=3)
    
    # 3. Save PSO History
    optimizer.save_results(f"initial_results.csv")
    
    # 4. Final Test
    print("\n" + "="*50 + "\nEVALUATING ROBUST BEST CONFIG ON TEST SET\n" + "="*50)
    print(f"\nRun {run_id + 1} - Robust Val: {optimizer.best_val_accuracy:.4%}")

    return optimizer.best_val_accuracy, optimizer.best_params, optimizer

if __name__ == "__main__":
    NUM_RUNS = 1  # Number of independent optimization runs
    STOCHASTICITY_SEED_BASE = SEED

    print("="*60)
    print(f"STOCHASTICITY REPORTING: {NUM_RUNS} FULL OPTIMIZATION RUN")
    print("="*60)

    all_results = []
    val_accuracies = []

    optimizer = None

    try:
        # 1. Prepare Data (once, reused for all runs)
        data_mgr = DataManager(MODEL_NAME)
        data_bundle = data_mgr.prepare_data()

        for run in range(NUM_RUNS):
            set_global_seed(STOCHASTICITY_SEED_BASE + run)

            try:
                val_acc, best_params, optimizer = run_single_optimization(run, data_bundle)

                all_results.append({
                    "run_id": run + 1,
                    "seed": STOCHASTICITY_SEED_BASE + run,
                    "val_accuracy": val_acc,
                    "learning_rate": best_params.learning_rate,
                    "warmup_ratio": best_params.warmup_ratio,
                    "rank": best_params.rank,
                    "alpha": best_params.alpha,
                    "dropout": best_params.dropout,
                    "target_modules": str(best_params.target_modules)
                })

                val_accuracies.append(val_acc)
                
            except Exception as e:
                print(f"!!! ERROR in Run {run + 1}: {e}")
                import traceback
                traceback.print_exc()
                cleanup_memory()

        # 3. Print Summary Statistics
        print("\n" + "="*60)
        print("STOCHASTICITY SUMMARY")
        print("="*60)

        if val_accuracies:
            val_mean = np.mean(val_accuracies)
            val_std = np.std(val_accuracies)
            val_min = np.min(val_accuracies)
            val_max = np.max(val_accuracies)

            print(f"\nVALIDATION ACCURACY (from PSO search):")
            print(f"  Mean: {val_mean:.4%}")
            print(f"  Std:  {val_std:.4%}")
            print(f"  Min:  {val_min:.4%}")
            print(f"  Max:  {val_max:.4%}")

            print(f"\nIndividual Results:")
            for result in all_results:
                print(f"  Run {result['run_id']}: Val={result['val_accuracy']:.4%}")

        # 4. Save stochasticity results
        results_df = pd.DataFrame(all_results)
        results_df.to_csv("pso_stochasticity_results.csv", index=False)
        print(f"\nStochasticity results saved to pso_stochasticity_results.csv")

        # Print Final Summary
        print("\n" + "="*60)
        print("FINAL SINGLE RUN SUMMARY")
        print("="*60)
        
        if all_results:
            res = all_results[0]
            print(f"Run 1 Results:")
            print(f"  Robust Validation Accuracy: {res['val_accuracy']:.4%}")
            print(f"  Best Parameters Found:")
            print(f"    LR: {res['learning_rate']:.2e}, Rank: {res['rank']}, Alpha: {res['alpha']}")

        # 4. Save results
        results_df = pd.DataFrame(all_results)
        results_df.to_csv("pso_final_results.csv", index=False)
        print(f"\nFinal results saved to pso_final_results.csv")
    
    except KeyboardInterrupt:
        print("\nOptimization interrupted by user.")
    except Exception as e:
        print(f"\nCritical failure: {e}")
        import traceback
        traceback.print_exc()
    finally:
        # Always save whatever results we have
        if optimizer is not None:
            print("Executing emergency save...")
            optimizer.save_results("pso_optimization_results.csv")
            
            if hasattr(optimizer, 'final_results') and optimizer.final_results:
                optimizer.save_top_solutions_results(
                    optimizer.final_results,
                    "pso_top_solutions_multiseed.csv"
                )
        cleanup_memory()
        print("Process Complete.")

STOCHASTICITY REPORTING: 1 FULL OPTIMIZATION RUN


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

...

Epoch,Training Loss,Validation Loss,Accuracy
1,1.077200,0.579559,0.798000
2,0.384200,0.371630,0.886500
3,0.216300,0.336501,0.892000


Acc: 89.2000%
   >>> Stats | Mean Acc: 89.6500% | Std: 0.0032

Top solutions results saved to robust_results.csv

BEST CONFIGURATION (Formatted Output):
Rank: 5.0
Mean Accuracy: 89.6500% ± 0.0032
Params: LR=2.00e-04, r=8.0, alpha=96.0
!!! ERROR in Run 1: name 'raw_robustness_data' is not defined


Traceback (most recent call last):
  File "/tmp/ipykernel_47/1756062320.py", line 50, in <cell line: 0>
    val_acc, best_params = run_single_optimization(run, data_bundle)
                           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/tmp/ipykernel_47/1756062320.py", line 19, in run_single_optimization
    optimizer.evaluate_top_solutions_with_seeds(run_id=run_id, num_top=5, num_seeds=3)
  File "/tmp/ipykernel_47/628461661.py", line 237, in evaluate_top_solutions_with_seeds
    raw_df = pd.DataFrame(raw_robustness_data)
                          ^^^^^^^^^^^^^^^^^^^
NameError: name 'raw_robustness_data' is not defined



STOCHASTICITY SUMMARY

Stochasticity results saved to pso_stochasticity_results.csv

FINAL SINGLE RUN SUMMARY

Final results saved to pso_final_results.csv


NameError: name 'optimizer' is not defined

# **7.   Plot Generation**

In [14]:
def visualize_results():
    # Target specifically Run 1 (The first full execution)
    file_path = "initial_results.csv"
    
    # if not os.path.exists(file_path):
    #     print(f"Results file '{file_path}' not found.")
    #     return
    
    # Load data
    df = pd.read_csv(file_path)
    
    # Ensure we are only looking at the Search Phase (Phase 1)
    # Filter out any robustness trials if they accidentally got mixed in (Trial IDs > 9000)
    if 'trial_id' in df.columns:
        df = df[df['trial_id'] < 9000].copy()
        
    print(f"Plotting Phase 1 Search Results: {len(df)} trials (20 particles x 5 epochs)")
    
    plt.style.use('seaborn-v0_8-whitegrid')
    fig, axes = plt.subplots(1, 3, figsize=(20,6))
    
    color = 'tab:blue'
    run_name = "Run 1 (Search Phase)"
    
    # --- Plot 1: Convergence Trend ---
    # Shows how the swarm found better solutions over the 80 trials
    df['best_so_far'] = df['val_accuracy'].cummax()
    
    axes[0].plot(df['trial_id'], df['best_so_far'], label='Best So Far', color='red', linewidth=2)
    axes[0].scatter(df['trial_id'], df['val_accuracy'], label='Individual Trials', color=color, alpha=0.5, s=15)
    
    # --- Plot 2: Accuracy Distribution ---
    # Shows the spread of performance among the 20 individuals over 4 epochs
    axes[1].hist(df['val_accuracy'], bins=15, alpha=0.7, label=run_name, color=color, edgecolor='black')
    
    # --- Plot 3: Training Time vs Accuracy ---
    # if 'training_time_sec' in df.columns:
    #     axes[2].scatter(df['training_time_sec'], df['val_accuracy'], 
    #                     label=run_name, color=color, alpha=0.6, edgecolors='w', s=40)
    # else:
    #     axes[2].text(0.5, 0.5, "Time data missing", ha='center')

    # --- Formatting ---
    axes[0].set_title("Search Convergence (Run 1)", fontsize=14, fontweight='bold')
    axes[0].set_xlabel("Trial ID (1-80)", fontsize=12)
    axes[0].set_ylabel("Validation Accuracy", fontsize=12)
    axes[0].legend()
    axes[0].grid(True, linestyle='--', alpha=0.7)

    axes[1].set_title("Accuracy Distribution (Run 1)", fontsize=14, fontweight='bold')
    axes[1].set_xlabel("Validation Accuracy", fontsize=12)
    axes[1].set_ylabel("Frequency", fontsize=12)
    
    axes[2].set_title("Training Cost vs. Performance (Run 1)", fontsize=14, fontweight='bold')
    # axes[2].set_xlabel("Training Time (s)", fontsize=12)
    axes[2].set_ylabel("Validation Accuracy", fontsize=12)
    axes[2].grid(True, linestyle='--', alpha=0.7)
    
    plt.tight_layout()
    plt.savefig('pso_visualization_run1.png')
    plt.show()

In [ ]:
visualize_results()